In [1]:
import cv2
from keras.models import load_model
import numpy as np
import os
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
model = load_model('emojinator.h5')

In [3]:
def main():
    emojis = get_emojis()
    cap = cv2.VideoCapture(0)
    x, y, w, h = 300, 50, 350, 350

    while (cap.isOpened()):
        ret, img = cap.read()
        img = cv2.flip(img, 1)
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask2 = cv2.inRange(hsv, np.array([2, 50, 60]), np.array([25, 150, 255]))
        res = cv2.bitwise_and(img, img, mask=mask2)
        gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
        median = cv2.GaussianBlur(gray, (5, 5), 0)

        kernel_square = np.ones((5, 5), np.uint8)
        dilation = cv2.dilate(median, kernel_square, iterations=2)
        opening = cv2.morphologyEx(dilation, cv2.MORPH_CLOSE, kernel_square)
        ret, thresh = cv2.threshold(opening, 30, 255, cv2.THRESH_BINARY)

        thresh = thresh[y:y + h, x:x + w]
        contours = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[0]
        if len(contours) > 0:
            contour = max(contours, key=cv2.contourArea)
            if cv2.contourArea(contour) > 2500:
                x, y, w1, h1 = cv2.boundingRect(contour)
                newImage = thresh[y:y + h1, x:x + w1]
                newImage = cv2.resize(newImage, (50, 50))
                pred_probab, pred_class = keras_predict(model, newImage)
                print(pred_class, pred_probab)
                img = overlay(img, emojis[pred_class], 400, 250, 90, 90)

        x, y, w, h = 300, 50, 350, 350
        cv2.imshow("Frame", img)
        cv2.imshow("Contours", thresh)
        k = cv2.waitKey(10)
        if k == 27:
            break


In [4]:
def keras_predict(model, image):
    processed = keras_process_image(image)
    pred_probab = model.predict(processed)[0]
    pred_class = list(pred_probab).index(max(pred_probab))
    return max(pred_probab), pred_class

In [5]:
def keras_process_image(img):
    image_x = 50
    image_y = 50
    img = cv2.resize(img, (image_x, image_y))
    img = np.array(img, dtype=np.float32)
    img = np.reshape(img, (-1, image_x, image_y, 1))
    return img

In [6]:
def get_emojis():
    emojis_folder = 'E:/ML internship/Emojinator/hand_emo/'
    emojis = []
    for emoji in range(len(os.listdir(emojis_folder))):
        print(emoji)
        emojis.append(cv2.imread(emojis_folder+str(emoji)+'.png', -1))
    return emojis

In [7]:
def overlay(image, emoji, x,y,w,h):
    emoji = cv2.resize(emoji, (w, h))
    try:
        image[y:y+h, x:x+w] = blend_transparent(image[y:y+h, x:x+w], emoji)
    except:
        pass
    return image


In [8]:
def blend_transparent(face_img, overlay_t_img):
    # Split out the transparency mask from the colour info
    overlay_img = overlay_t_img[:,:,:3] # Grab the BRG planes
    overlay_mask = overlay_t_img[:,:,3:]  # And the alpha plane

    # Again calculate the inverse mask
    background_mask = 255 - overlay_mask

    # Turn the masks into three channel, so we can use them as weights
    overlay_mask = cv2.cvtColor(overlay_mask, cv2.COLOR_GRAY2BGR)
    background_mask = cv2.cvtColor(background_mask, cv2.COLOR_GRAY2BGR)

    # Create a masked out face image, and masked out overlay
    # We convert the images to floating point in range 0.0 - 1.0
    face_part = (face_img * (1 / 255.0)) * (background_mask * (1 / 255.0))
    overlay_part = (overlay_img * (1 / 255.0)) * (overlay_mask * (1 / 255.0))

    # And finally just add them together, and rescale it back to an 8bit integer image
    return np.uint8(cv2.addWeighted(face_part, 255.0, overlay_part, 255.0, 0.0))


In [9]:
keras_predict(model, np.zeros((50, 50, 1), dtype=np.uint8))

(0.8095797, 1)

In [10]:
main()

0
1
2
3
4
5
6
7
8
9
10
11
12
3 0.44886398
9 0.55439913
3 0.63164735
3 0.47505334
3 0.7493629
3 0.45380014
3 0.80322313
3 0.4074381
9 0.9933575
3 0.68584687
3 0.6293137
3 0.79549307
3 0.91139114
9 0.28437757
3 0.35252744
3 0.4234846
3 0.34292904
3 0.4951214
3 0.72624093
3 0.5911397
9 0.9950805
3 0.30502367
3 0.7936016
3 0.27673876
4 0.23154943
9 0.6128942
3 0.43013218
3 0.3011984
1 0.23498684
3 0.36585924
4 0.3301856
9 0.33464986
3 0.27329198
9 0.5273873
3 0.5503831
3 0.47134012
3 0.35021344
4 0.33839166
9 0.40520248
9 0.56610876
3 0.40209296
9 0.7153224
3 0.36853477
4 0.3347918
3 0.37738958
4 0.3327631
9 0.5348934
3 0.41764513
4 0.590851
4 0.2950349
3 0.29189187
4 0.49853346
4 0.60867995
9 0.32249638
4 0.75951624
9 0.73108405
9 0.44926798
4 0.59580576
11 0.50522465
3 0.37296554
9 0.79848737
4 0.3988525
3 0.53009593
11 0.51633334
4 0.73034906
4 0.4650588
4 0.5750326
4 0.5750326
9 0.3867372
5 0.2717756
3 0.4974377
3 0.40919194
11 0.55289805
5 0.7520318
11 0.92465544
5 0.9997501
11 0.6734